# 🌡️ Professional Sensor Analytics Dashboard

**Production-grade IoT monitoring with real-time data sync and advanced analytics**

Built with best practices for data visualization, accessibility, and user experience.

In [1]:
# Install required packages
!pip install firebase-admin gradio plotly scipy scikit-learn pandas numpy requests gdown -q

In [2]:
# Download Firebase credentials from public Google Drive
import gdown
import os

firebase_key_file = 'firebase_key.json'
print('📥 Downloading Firebase credentials...')

try:
    FILE_ID = '15L_nwwjOXYZ1DwTZUc6Xk2oB7LH5lmSa'
    url = f'https://drive.google.com/uc?id={FILE_ID}'
    gdown.download(url, firebase_key_file, quiet=False)
    print('✓ Firebase credentials ready')
except Exception as e:
    print(f'⚠️ Download failed: {e}')
    print('Use manual upload below')

📥 Downloading Firebase credentials...


Downloading...
From: https://drive.google.com/uc?id=15L_nwwjOXYZ1DwTZUc6Xk2oB7LH5lmSa
To: /content/firebase_key.json
100%|██████████| 2.37k/2.37k [00:00<00:00, 6.99MB/s]

✓ Firebase credentials ready


In [3]:
# ALTERNATIVE: Manual upload
from google.colab import files
import os

if not os.path.exists('firebase_key.json'):
    print('📤 Upload firebase_key.json manually:')
    uploaded = files.upload()
    if uploaded:
        print('✓ File uploaded successfully!')
else:
    print('✓ Firebase key already available')

✓ Firebase key already available


In [4]:
# Import all required libraries
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import gradio as gr
from datetime import datetime, timedelta
from scipy import stats
from sklearn.linear_model import LinearRegression
import requests
import json
import warnings
warnings.filterwarnings('ignore')

# Initialize Firebase
firebase_key_file = 'firebase_key.json'

if not firebase_admin._apps:
    cred = credentials.Certificate(firebase_key_file)
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://cloud-81451-default-rtdb.europe-west1.firebasedatabase.app/'
    })
    print('✓ Firebase initialized successfully')
else:
    print('✓ Firebase already initialized')

# API Configuration
BASE_URL = "https://server-cloud-v645.onrender.com/"
FEED = "json"
BATCH_LIMIT = 200

print('✓ All packages loaded successfully!')

✓ Firebase initialized successfully
✓ All packages loaded successfully!


In [5]:
# ============== DESIGN SYSTEM & COLOR PALETTES ==============

# Okabe-Ito colorblind-safe palette
COLORS = {
    'temperature': {
        'primary': '#ef4444',      # Red
        'gradient': 'linear-gradient(135deg, #dc2626 0%, #ef4444 100%)',
        'scale': ['#3b82f6', '#60a5fa', '#93c5fd', '#fef3c7', '#fde047', '#fb923c', '#ef4444', '#dc2626']  # Blue to Red
    },
    'humidity': {
        'primary': '#3b82f6',      # Blue
        'gradient': 'linear-gradient(135deg, #2563eb 0%, #3b82f6 100%)',
        'scale': ['#f0f9ff', '#e0f2fe', '#bae6fd', '#7dd3fc', '#38bdf8', '#0ea5e9', '#0284c7', '#0369a1']  # Light to Dark Blue
    },
    'soil': {
        'primary': '#10b981',      # Green
        'gradient': 'linear-gradient(135deg, #059669 0%, #10b981 100%)',
        'scale': ['#fef3c7', '#fde68a', '#fcd34d', '#a3e635', '#84cc16', '#65a30d', '#4d7c0f', '#3f6212']  # Yellow to Green
    },
    'status': {
        'normal': '#10b981',       # Green
        'warning': '#f59e0b',      # Yellow
        'critical': '#ef4444',     # Red
        'info': '#3b82f6',         # Blue
        'inactive': '#9ca3af'      # Gray
    },
    'neutral': {
        'text': '#1f2937',
        'subtext': '#6b7280',
        'border': '#e5e7eb',
        'bg': '#ffffff',
        'bg_secondary': '#f9fafb'
    }
}

# Typography scale (px)
TYPOGRAPHY = {
    'title': '32px',           # Dashboard title
    'section': '24px',         # Section headers
    'card_title': '14px',      # Card labels
    'metric': '48px',          # Large metric values
    'body': '16px',            # Body text
    'small': '14px',           # Small text
    'tiny': '12px'             # Axis labels
}

# Spacing scale (px) - 8px grid system
SPACING = {
    'xs': '4px',
    'sm': '8px',
    'md': '16px',
    'lg': '24px',
    'xl': '32px',
    'xxl': '48px'
}

# Material Design elevation shadows
SHADOWS = {
    'sm': '0 1px 3px rgba(0,0,0,0.12), 0 1px 2px rgba(0,0,0,0.24)',
    'md': '0 3px 6px rgba(0,0,0,0.16), 0 3px 6px rgba(0,0,0,0.23)',
    'lg': '0 10px 20px rgba(0,0,0,0.19), 0 6px 6px rgba(0,0,0,0.23)'
}

print('✓ Design system loaded')

✓ Design system loaded


In [6]:
# ============== FIREBASE SYNC FUNCTIONS ==============

def get_latest_timestamp_from_firebase():
    """Get the most recent timestamp from Firebase"""
    try:
        ref = db.reference('/sensor_data')
        latest = ref.order_by_child('created_at').limit_to_last(1).get()
        if latest:
            return latest[list(latest.keys())[0]]['created_at']
    except:
        pass
    return None

def fetch_batch_from_server(before_timestamp=None):
    """Fetch data batch from server API"""
    params = {"feed": FEED, "limit": BATCH_LIMIT}
    if before_timestamp:
        params["before_created_at"] = before_timestamp
    try:
        return requests.get(f"{BASE_URL}/history", params=params, timeout=180).json()
    except:
        return {}

def save_to_firebase(data_list):
    """Save data records to Firebase"""
    if not data_list:
        return 0
    ref = db.reference('/sensor_data')
    saved = 0
    for sample in data_list:
        try:
            key = sample['created_at'].replace(':', '-').replace('.', '-')
            vals = json.loads(sample['value'])
            ref.child(key).set({
                'created_at': sample['created_at'],
                'temperature': vals['temperature'],
                'humidity': vals['humidity'],
                'soil': vals['soil']
            })
            saved += 1
        except:
            continue
    return saved

def sync_new_data_from_server():
    """Sync new data from server to Firebase"""
    msgs = ["🔄 Starting sync..."]
    latest = get_latest_timestamp_from_firebase()

    if latest:
        msgs.append(f"📊 Latest data: {latest}")
    else:
        msgs.append("📭 No existing data in Firebase")

    msgs.append("🌐 Fetching from server...")
    resp = fetch_batch_from_server()

    if "data" not in resp:
        msgs.append("❌ Error fetching data from server")
        return "\n".join(msgs), 0

    new = [s for s in resp["data"] if not latest or s["created_at"] > latest]

    if new:
        msgs.append(f"✨ Found {len(new)} new samples")
        msgs.append("💾 Saving to Firebase...")
        saved = save_to_firebase(new)
        msgs.append(f"✅ Successfully saved {saved} records!")
        return "\n".join(msgs), saved

    msgs.append("✓ No new data. Database is up to date!")
    return "\n".join(msgs), 0

print('✓ Sync functions loaded')

✓ Sync functions loaded


In [7]:
# ============== DATA LOADING ==============

def load_data_from_firebase():
    """Load all sensor data from Firebase"""
    ref = db.reference('/sensor_data')
    data = ref.get()
    if not data:
        return pd.DataFrame()

    records = []
    for key, value in data.items():
        records.append({
            'timestamp': pd.to_datetime(value['created_at']),
            'temperature': float(value['temperature']),
            'humidity': float(value['humidity']),
            'soil': float(value['soil'])
        })

    df = pd.DataFrame(records).sort_values('timestamp').reset_index(drop=True)
    return df

print('📥 Loading initial data from Firebase...')
df = load_data_from_firebase()
print(f'✓ Loaded {len(df)} records')
if len(df) > 0:
    print(f'📅 Date range: {df["timestamp"].min()} to {df["timestamp"].max()}')
df.head()

📥 Loading initial data from Firebase...
✓ Loaded 600 records
📅 Date range: 2025-12-10 05:23:39+00:00 to 2025-12-14 09:14:21+00:00


,timestamp,temperature,humidity,soil
0,2025-12-10 05:23:39+00:00,18.1,54.0,66.0
1,2025-12-10 05:33:39+00:00,18.0,55.0,65.0
2,2025-12-10 05:43:39+00:00,18.0,55.0,66.0
3,2025-12-10 05:53:39+00:00,18.2,55.0,66.0
4,2025-12-10 06:03:39+00:00,18.6,55.0,65.0


In [8]:
# ============== CUSTOM CSS STYLING ==============

CUSTOM_CSS = f"""
/* Import Inter font for professional typography */
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');

/* Global styles */
* {{
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
}}

/* KPI Metric Cards */
.kpi-card {{
    background: {COLORS['neutral']['bg']};
    padding: {SPACING['lg']};
    border-radius: 12px;
    box-shadow: {SHADOWS['sm']};
    text-align: center;
    transition: transform 0.2s, box-shadow 0.2s;
    border-left: 4px solid;
}}

.kpi-card:hover {{
    transform: translateY(-4px);
    box-shadow: {SHADOWS['md']};
}}

.kpi-label {{
    color: {COLORS['neutral']['subtext']};
    font-size: {TYPOGRAPHY['card_title']};
    font-weight: 600;
    text-transform: uppercase;
    letter-spacing: 0.5px;
    margin: 0 0 {SPACING['sm']} 0;
}}

.kpi-value {{
    font-size: {TYPOGRAPHY['metric']};
    font-weight: 700;
    margin: {SPACING['sm']} 0;
    color: {COLORS['neutral']['text']};
    line-height: 1;
}}

.kpi-change {{
    font-size: {TYPOGRAPHY['small']};
    font-weight: 600;
    display: inline-flex;
    align-items: center;
    gap: 4px;
}}

.trend-up {{ color: {COLORS['status']['normal']}; }}
.trend-down {{ color: {COLORS['status']['critical']}; }}
.trend-stable {{ color: {COLORS['status']['info']}; }}

/* Stat Cards with Gradients */
.stat-card {{
    border-radius: 16px;
    padding: {SPACING['lg']};
    color: white;
    box-shadow: {SHADOWS['md']};
    margin: {SPACING['md']} 0;
    transition: transform 0.2s, box-shadow 0.2s;
}}

.stat-card:hover {{
    transform: translateY(-2px);
    box-shadow: {SHADOWS['lg']};
}}

.stat-card h2 {{
    margin: 0 0 {SPACING['md']} 0;
    font-size: {TYPOGRAPHY['section']};
    font-weight: 600;
}}

.stat-item {{
    background: rgba(255,255,255,0.15);
    border-radius: 12px;
    padding: {SPACING['md']};
    backdrop-filter: blur(10px);
    transition: background 0.2s;
}}

.stat-item:hover {{
    background: rgba(255,255,255,0.25);
}}

/* Tooltip System */
.info-popup {{
    position: relative;
    display: inline-block;
}}

.info-text {{
    visibility: hidden;
    width: 280px;
    background-color: rgba(0, 0, 0, 0.95);
    color: white !important;
    text-align: left;
    border-radius: 8px;
    padding: 14px;
    position: absolute;
    z-index: 10000;
    right: calc(100% + 10px);
    top: 50%;
    transform: translateY(-50%);
    font-size: 13px;
    line-height: 1.6;
    box-shadow: 0 6px 20px rgba(0,0,0,0.5);
    white-space: normal;
    word-wrap: break-word;
    pointer-events: none;
}}

.info-popup:hover .info-text {{
    visibility: visible;
}}

.info-text::after {{
    content: "";
    position: absolute;
    left: 100%;
    top: 50%;
    transform: translateY(-50%);
    border: 8px solid transparent;
    border-left-color: rgba(0, 0, 0, 0.95);
}}

/* Status Badge */
.status-badge {{
    display: inline-flex;
    align-items: center;
    gap: 6px;
    padding: 6px 16px;
    border-radius: 20px;
    font-size: {TYPOGRAPHY['small']};
    font-weight: 600;
    background: #dcfce7;
    color: #166534;
}}

.status-dot {{
    width: 8px;
    height: 8px;
    border-radius: 50%;
    background: #10b981;
    animation: pulse 2s infinite;
}}

@keyframes pulse {{
    0%, 100% {{ opacity: 1; }}
    50% {{ opacity: 0.5; }}
}}

/* Explanation Cards */
.explanation-card {{
    background: {COLORS['temperature']['gradient']};
    border-radius: 12px;
    padding: {SPACING['md']};
    color: white;
    margin-bottom: {SPACING['md']};
    box-shadow: {SHADOWS['sm']};
}}

.explanation-card h3 {{
    margin: 0 0 {SPACING['sm']} 0;
    font-size: 18px;
    font-weight: 600;
}}

.explanation-card p {{
    margin: 0 0 {SPACING['xs']} 0;
    font-size: {TYPOGRAPHY['small']};
    opacity: 0.95;
    line-height: 1.5;
}}

/* Tab styling */
.tab-nav button {{
    font-weight: 500;
    font-size: {TYPOGRAPHY['body']};
}}
"""

print('✓ CSS styling loaded')

✓ CSS styling loaded


In [9]:
# ============== METRIC CARD COMPONENTS ==============

def create_kpi_card(label, value, unit, change_value, change_label, trend="up", border_color=None):
    """Create a professional KPI metric card"""
    if border_color is None:
        border_color = COLORS['status']['normal']

    trend_class = f"trend-{trend}"
    trend_icon = "↑" if trend == "up" else ("↓" if trend == "down" else "→")

    return f"""
    <div class="kpi-card" style="border-left-color: {border_color};">
        <p class="kpi-label">{label}</p>
        <p class="kpi-value">{value}<span style="font-size: 24px; font-weight: 500;">{unit}</span></p>
        <p class="kpi-change {trend_class}">
            <span>{trend_icon}</span>
            <span>{change_value} {change_label}</span>
        </p>
    </div>
    """

def create_status_badge(text="LIVE", pulse=True):
    """Create a live status indicator badge"""
    dot = '<span class="status-dot"></span>' if pulse else ''
    return f'<span class="status-badge">{dot}{text}</span>'

def create_explanation_card(title, description, interpretation, color_gradient=None):
    """Create an explanation card for visualizations"""
    if color_gradient is None:
        color_gradient = COLORS['temperature']['gradient']

    return f"""
    <div class="explanation-card" style="background: {color_gradient};">
        <h3>📊 {title}</h3>
        <p><strong>What it shows:</strong> {description}</p>
        <p><strong>How to interpret:</strong> {interpretation}</p>
    </div>
    """

print('✓ Component functions loaded')

✓ Component functions loaded


In [10]:
# ============== STATISTICS CARDS WITH TOOLTIPS ==============

def create_stat_cards_html(df):
    """Create beautiful statistics cards with tooltips"""
    if len(df) == 0:
        return "<p>No data available</p>"

    stats = {}
    for col in ['temperature', 'humidity', 'soil']:
        stats[col] = {
            'Mean': round(df[col].mean(), 2),
            'Median': round(df[col].median(), 2),
            'Std Dev': round(df[col].std(), 2),
            'Min': round(df[col].min(), 2),
            'Max': round(df[col].max(), 2),
            'Q25': round(df[col].quantile(0.25), 2),
            'Q75': round(df[col].quantile(0.75), 2),
            'IQR': round(df[col].quantile(0.75) - df[col].quantile(0.25), 2)
        }

    explanations = {
        'Mean': 'Average value across all measurements. Sum of all values ÷ number of measurements.',
        'Median': 'Middle value when sorted. 50% of values are below, 50% above.',
        'Std Dev': 'How much values vary around the mean. Low = consistent, High = variable.',
        'Min': 'Lowest value recorded during the measurement period.',
        'Max': 'Highest value recorded during the measurement period.',
        'Q25': 'First quartile (25th percentile). 25% of values are below this.',
        'Q75': 'Third quartile (75th percentile). 75% of values are below this.',
        'IQR': 'Interquartile range (Q75-Q25). Middle 50% of values fall within this range.'
    }

    colors = {
        'temperature': COLORS['temperature']['gradient'],
        'humidity': COLORS['humidity']['gradient'],
        'soil': COLORS['soil']['gradient']
    }

    units = {'temperature': '°C', 'humidity': '%', 'soil': '%'}
    names = {'temperature': 'TEMPERATURE', 'humidity': 'HUMIDITY', 'soil': 'SOIL MOISTURE'}

    html = '<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 16px; max-width: 1400px; margin: 0 auto;">'

    for var in ['temperature', 'humidity', 'soil']:
        html += f"""
        <div class="stat-card" style="background: {colors[var]};">
            <h2>{names[var]}</h2>
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 14px;">
        """

        for stat_name, stat_value in stats[var].items():
            html += f"""
            <div class="stat-item">
                <div style="display: flex; justify-content: space-between; margin-bottom: 8px;">
                    <div style="font-size: 13px; opacity: 0.95; font-weight: 500;">{stat_name}</div>
                    <div class="info-popup"
                         onmouseenter="this.positionTooltip(event)"
                         style="
                        width: 20px; height: 20px; border-radius: 50%;
                        border: 2px solid rgba(255,255,255,0.7);
                        display: flex; align-items: center; justify-content: center;
                        font-size: 13px; font-weight: bold; cursor: help;
                        background: rgba(255,255,255,0.1);">
                        i<span class="info-text">{explanations[stat_name]}</span>
                    </div>
                </div>
                <div style="font-size: 26px; font-weight: 700;">{stat_value}{units[var]}</div>
            </div>
            """

        html += "</div></div>"

    html += "</div>"

    # Add inline JavaScript for tooltip positioning
    html += """
<script>
(function() {
    // Wait for DOM to be ready
    setTimeout(function() {
        document.querySelectorAll('.info-popup').forEach(function(popup) {
            popup.positionTooltip = function(event) {
                const tooltip = this.querySelector('.info-text');
                if (!tooltip) return;

                const rect = this.getBoundingClientRect();
                const tooltipWidth = 280;
                const gap = 15;
                const viewportWidth = window.innerWidth;
                const viewportHeight = window.innerHeight;

                // Calculate position
                let left, top;

                // Horizontal positioning
                if (rect.left > tooltipWidth + gap) {
                    // Show on left
                    left = rect.left - tooltipWidth - gap;
                } else if (rect.right + tooltipWidth + gap < viewportWidth) {
                    // Show on right
                    left = rect.right + gap;
                } else {
                    // Center on screen
                    left = (viewportWidth - tooltipWidth) / 2;
                }

                // Vertical positioning (centered on icon)
                top = rect.top + (rect.height / 2);

                // Apply positioning
                tooltip.style.left = left + 'px';
                tooltip.style.top = top + 'px';
                tooltip.style.transform = 'translateY(-50%)';
            };
        });
    }, 100);
})();
</script>
"""

    return html

print('✓ Statistics card functions loaded')

✓ Statistics card functions loaded


In [11]:
# ============== PLOTLY CHART FUNCTIONS WITH BEST PRACTICES ==============

# Global Plotly template configuration
PLOTLY_TEMPLATE = {
    'layout': go.Layout(
        font=dict(family="Inter, sans-serif", size=14, color=COLORS['neutral']['text']),
        plot_bgcolor='white',
        paper_bgcolor='white',
        hovermode='x unified',
        hoverlabel=dict(
            bgcolor="white",
            font_size=13,
            font_family="Inter"
        ),
        margin=dict(l=60, r=30, t=80, b=60)
    )
}

def apply_chart_styling(fig, title="", xaxis_title="", yaxis_title="", height=400):
    """Apply consistent styling to all charts"""
    fig.update_layout(
        title=dict(text=title, font=dict(size=20, weight=600)),
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        font=dict(family="Inter, sans-serif", size=14),
        plot_bgcolor='white',
        paper_bgcolor='white',
        height=height,
        hovermode='x unified'
    )

    fig.update_xaxes(
        showgrid=False,
        title_font=dict(size=14, color=COLORS['neutral']['subtext']),
        tickfont=dict(size=12)
    )

    fig.update_yaxes(
        showgrid=True,
        gridcolor='#E5E7EB',
        title_font=dict(size=14, color=COLORS['neutral']['subtext']),
        tickfont=dict(size=12)
    )

    return fig

def time_series_overview(df):
    """Main time series plot with proper styling"""
    fig = go.Figure()

    # Temperature with red color scale
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['temperature'],
        name='Temperature',
        mode='lines',
        line=dict(color=COLORS['temperature']['primary'], width=2),
        hovertemplate='%{y:.1f}°C<extra></extra>'
    ))

    # Humidity with blue color scale
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['humidity'],
        name='Humidity',
        mode='lines',
        line=dict(color=COLORS['humidity']['primary'], width=2),
        hovertemplate='%{y:.1f}%<extra></extra>'
    ))

    # Soil with green color scale
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['soil'],
        name='Soil Moisture',
        mode='lines',
        line=dict(color=COLORS['soil']['primary'], width=2),
        hovertemplate='%{y:.1f}%<extra></extra>'
    ))

    apply_chart_styling(
        fig,
        title="Sensor Data Time Series",
        xaxis_title="Time",
        yaxis_title="Value",
        height=500
    )

    # Add mean reference lines
    for col, color in [('temperature', COLORS['temperature']['primary']),
                       ('humidity', COLORS['humidity']['primary']),
                       ('soil', COLORS['soil']['primary'])]:
        mean_val = df[col].mean()
        fig.add_hline(
            y=mean_val,
            line_dash="dash",
            line_color=color,
            opacity=0.3,
            annotation_text=f"{col.capitalize()} Mean",
            annotation_position="right"
        )

    explanation = create_explanation_card(
        "Time Series Overview",
        "All three sensor measurements plotted over time, showing how values change chronologically. Mean reference lines (dashed) show average values.",
        "Look for trends (increasing/decreasing over time), cycles (repeating patterns), and sudden changes (events or sensor issues).",
        COLORS['temperature']['gradient']
    )

    return explanation, fig

def calculate_correlations(df):
    """Correlation matrix with RdBu diverging colormap"""
    corr_matrix = df[['temperature', 'humidity', 'soil']].corr()

    fig = px.imshow(
        corr_matrix,
        labels=dict(color="Correlation"),
        x=['Temperature', 'Humidity', 'Soil'],
        y=['Temperature', 'Humidity', 'Soil'],
        color_continuous_scale='RdBu_r',  # Red-Blue diverging
        zmin=-1, zmax=1,
        aspect="auto"
    )

    apply_chart_styling(
        fig,
        title="Correlation Matrix",
        xaxis_title="Variables",
        yaxis_title="Variables",
        height=500
    )

    # Add correlation values as text
    for i in range(len(corr_matrix)):
        for j in range(len(corr_matrix)):
            fig.add_annotation(
                x=j, y=i,
                text=str(round(corr_matrix.iloc[i, j], 3)),
                showarrow=False,
                font=dict(
                    size=14,
                    color='black' if abs(corr_matrix.iloc[i, j]) < 0.5 else 'white',
                    weight=600
                )
            )

    explanation = create_explanation_card(
        "Correlation Analysis",
        "The strength and direction of linear relationships between temperature, humidity, and soil moisture. Values range from -1 (perfect negative) to +1 (perfect positive).",
        "Values close to +1: variables increase together. Close to -1: one increases as other decreases. Near 0: no linear relationship.",
        COLORS['humidity']['gradient']
    )

    return explanation, fig

def hourly_patterns(df):
    """Hourly patterns with proper axis labels"""
    df_copy = df.copy()
    df_copy['hour'] = df_copy['timestamp'].dt.hour
    hourly_avg = df_copy.groupby('hour')[['temperature', 'humidity', 'soil']].mean()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=hourly_avg.index,
        y=hourly_avg['temperature'],
        name='Temperature',
        mode='lines+markers',
        line=dict(color=COLORS['temperature']['primary'], width=2.5),
        marker=dict(size=8)
    ))

    fig.add_trace(go.Scatter(
        x=hourly_avg.index,
        y=hourly_avg['humidity'],
        name='Humidity',
        mode='lines+markers',
        line=dict(color=COLORS['humidity']['primary'], width=2.5),
        marker=dict(size=8)
    ))

    fig.add_trace(go.Scatter(
        x=hourly_avg.index,
        y=hourly_avg['soil'],
        name='Soil',
        mode='lines+markers',
        line=dict(color=COLORS['soil']['primary'], width=2.5),
        marker=dict(size=8)
    ))

    apply_chart_styling(
        fig,
        title="Average Values by Hour of Day",
        xaxis_title="Hour of Day (0-23)",
        yaxis_title="Average Value",
        height=450
    )

    explanation = create_explanation_card(
        "Hourly Patterns",
        "Average sensor values for each hour of the day, revealing daily cycles and patterns in temperature, humidity, and soil moisture.",
        "Look for peaks and valleys that repeat daily. Temperature typically peaks in afternoon hours. Humidity often shows inverse patterns.",
        COLORS['soil']['gradient']
    )

    return explanation, fig

def daily_patterns(df):
    """Daily patterns with min/max ranges"""
    df_copy = df.copy()
    df_copy['date'] = df_copy['timestamp'].dt.date
    daily = df_copy.groupby('date')[['temperature', 'humidity', 'soil']].agg(['mean', 'min', 'max'])

    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('Temperature (°C)', 'Humidity (%)', 'Soil Moisture (%)'),
        vertical_spacing=0.08
    )

    colors = {
        'temperature': COLORS['temperature']['primary'],
        'humidity': COLORS['humidity']['primary'],
        'soil': COLORS['soil']['primary']
    }

    for idx, var in enumerate(['temperature', 'humidity', 'soil'], 1):
        dates = [str(d) for d in daily.index]

        # Min-Max fill area
        fig.add_trace(go.Scatter(
            x=dates, y=daily[var]['max'],
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ), row=idx, col=1)

        fig.add_trace(go.Scatter(
            x=dates, y=daily[var]['min'],
            mode='lines',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=f"rgba({int(colors[var][1:3], 16)}, {int(colors[var][3:5], 16)}, {int(colors[var][5:7], 16)}, 0.2)",
            showlegend=False,
            hoverinfo='skip'
        ), row=idx, col=1)

        # Mean line
        fig.add_trace(go.Scatter(
            x=dates, y=daily[var]['mean'],
            mode='lines+markers',
            line=dict(color=colors[var], width=2.5),
            marker=dict(size=6),
            name='Mean',
            showlegend=(idx==1)
        ), row=idx, col=1)

    fig.update_xaxes(title_text="Date", row=3, col=1)
    fig.update_layout(height=900, showlegend=True)

    explanation = create_explanation_card(
        "Daily Trends",
        "Daily mean values (line) with min-max range (shaded area). The shaded region shows daily variability.",
        "Wider shaded areas indicate more variable conditions. Upward/downward trends show multi-day changes. Look for unusual days.",
        COLORS['temperature']['gradient']
    )

    return explanation, fig

def distribution_analysis(df):
    """Distribution histograms with proper bins"""
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Temperature (°C)', 'Humidity (%)', 'Soil Moisture (%)')
    )

    fig.add_trace(go.Histogram(
        x=df['temperature'],
        marker_color=COLORS['temperature']['primary'],
        nbinsx=30,
        name='Temperature'
    ), row=1, col=1)

    fig.add_trace(go.Histogram(
        x=df['humidity'],
        marker_color=COLORS['humidity']['primary'],
        nbinsx=30,
        name='Humidity'
    ), row=1, col=2)

    fig.add_trace(go.Histogram(
        x=df['soil'],
        marker_color=COLORS['soil']['primary'],
        nbinsx=30,
        name='Soil'
    ), row=1, col=3)

    # Set explicit axis ranges with small margin to prevent bars from exceeding limits
    fig.update_xaxes(title_text="Temperature (°C)", range=[-52, 102], row=1, col=1)
    fig.update_xaxes(title_text="Humidity (%)", range=[-2, 102], row=1, col=2)
    fig.update_xaxes(title_text="Soil Moisture (%)", range=[-2, 102], row=1, col=3)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)

    fig.update_layout(height=400, showlegend=False)

    explanation = create_explanation_card(
        "Distribution Analysis",
        "Frequency distribution showing how often each value range occurs. Reveals the shape of the data distribution.",
        "Bell-shaped curves indicate normal distribution. Multiple peaks suggest different operating modes. Skewed distributions show bias.",
        COLORS['humidity']['gradient']
    )

    return explanation, fig

def scatter_analysis(df):
    """Scatter plots with optimal opacity for overplotting"""
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Temp vs Humidity', 'Temp vs Soil', 'Humidity vs Soil')
    )

    fig.add_trace(go.Scatter(
        x=df['temperature'], y=df['humidity'],
        mode='markers',
        marker=dict(size=6, opacity=0.4, color=COLORS['temperature']['primary']),
        name='Temp-Humidity'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df['temperature'], y=df['soil'],
        mode='markers',
        marker=dict(size=6, opacity=0.4, color=COLORS['humidity']['primary']),
        name='Temp-Soil'
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=df['humidity'], y=df['soil'],
        mode='markers',
        marker=dict(size=6, opacity=0.4, color=COLORS['soil']['primary']),
        name='Humidity-Soil'
    ), row=1, col=3)

    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_xaxes(title_text="Temperature (°C)", row=1, col=2)
    fig.update_xaxes(title_text="Humidity (%)", row=1, col=3)
    fig.update_yaxes(title_text="Humidity (%)", row=1, col=1)
    fig.update_yaxes(title_text="Soil (%)", row=1, col=2)
    fig.update_yaxes(title_text="Soil (%)", row=1, col=3)

    fig.update_layout(height=400, showlegend=False)

    explanation = create_explanation_card(
        "Scatter Plot Analysis",
        "Point plots showing relationships between pairs of variables. Each dot represents one measurement.",
        "Linear patterns indicate correlation. Clustered points show common value combinations. Scattered points suggest independence.",
        COLORS['soil']['gradient']
    )

    return explanation, fig

def time_series_decomposition(df, variable='temperature'):
    """Moving averages with different window sizes"""
    df_s = df.sort_values('timestamp').copy()
    df_s['MA_3'] = df_s[variable].rolling(3, center=True).mean()
    df_s['MA_10'] = df_s[variable].rolling(10, center=True).mean()
    df_s['MA_30'] = df_s[variable].rolling(30, center=True).mean()

    fig = go.Figure()

    # Original data - more visible
    fig.add_trace(go.Scatter(
        x=df_s['timestamp'], y=df_s[variable],
        name='Raw Data',
        mode='lines',
        line=dict(width=1, color='#4B5563'),
        opacity=0.6
    ))

    # Moving averages with increasing line width
    fig.add_trace(go.Scatter(
        x=df_s['timestamp'], y=df_s['MA_3'],
        name='MA-3 (Short-term)',
        line=dict(width=1.5, color='#10b981')
    ))

    fig.add_trace(go.Scatter(
        x=df_s['timestamp'], y=df_s['MA_10'],
        name='MA-10 (Medium-term)',
        line=dict(width=2.5, color='#3b82f6')
    ))

    fig.add_trace(go.Scatter(
        x=df_s['timestamp'], y=df_s['MA_30'],
        name='MA-30 (Long-term)',
        line=dict(width=3.5, color='#ef4444')
    ))

    unit = '°C' if variable == 'temperature' else '%'
    apply_chart_styling(
        fig,
        title=f'Moving Averages - {variable.capitalize()}',
        xaxis_title='Time',
        yaxis_title=f'{variable.capitalize()} ({unit})',
        height=450
    )

    explanation = create_explanation_card(
        "Moving Averages",
        "Smoothed trend lines at different time scales (3, 10, 30 measurements). Line thickness increases with window size.",
        "MA-3 follows rapid changes. MA-10 shows medium-term trends. MA-30 reveals long-term patterns by filtering noise.",
        COLORS['temperature']['gradient']
    )

    return explanation, fig

def anomaly_detection(df):
    """Z-score anomaly detection"""
    df_copy = df.copy()
    anomalies = pd.DataFrame()

    for col in ['temperature', 'humidity', 'soil']:
        z = np.abs(stats.zscore(df_copy[col]))
        df_copy[f'{col}_anomaly'] = z > 3
        anom = df_copy[df_copy[f'{col}_anomaly']].copy()

        if len(anom) > 0:
            anom['variable'] = col
            anom['z_score'] = z[df_copy[f'{col}_anomaly']]
            anomalies = pd.concat([anomalies, anom[['timestamp', 'variable', col, 'z_score']]])

    if len(anomalies) > 0:
        anomalies = anomalies.sort_values('timestamp').reset_index(drop=True)
        for col in anomalies.select_dtypes(include=[np.number]).columns:
            anomalies[col] = anomalies[col].round(3)
        return anomalies

    return pd.DataFrame({'Message': ['No anomalies detected (Z-score > 3)']})

print('✓ All plot functions loaded with best practices')

✓ All plot functions loaded with best practices


In [12]:
# ============== GRADIO DASHBOARD ==============

def create_dashboard():
    """Create the complete professional dashboard"""

    with gr.Blocks(css=CUSTOM_CSS, theme=gr.themes.Soft(), title="Sensor Analytics Dashboard") as demo:

        # Header
        gr.Markdown(
            """# 🌡️ Environmental Sensor Analytics Dashboard
            ### Real-time monitoring with advanced statistical analysis
            """
        )

        with gr.Row():
            gr.HTML(create_status_badge("LIVE", pulse=True))
            data_info = gr.Markdown(
                f"**📊 Current Data:** {len(df)} records" +
                (f" | **📅 Range:** {df['timestamp'].min()} to {df['timestamp'].max()}" if len(df) > 0 else "")
            )

        # Control buttons
        with gr.Row():
            sync_btn = gr.Button("🔄 Sync & Refresh All Data", variant="primary", size="lg", scale=2)
            refresh_btn = gr.Button("↻ Refresh Dashboard", variant="secondary", scale=1)

        sync_status = gr.Textbox(label="📡 Sync Status", lines=6, interactive=False)

        # KPI Cards Row - Z-pattern layout (most important top-left)
        gr.Markdown("## 📈 Key Performance Indicators")

        with gr.Row(equal_height=True):
            if len(df) > 0:
                temp_current = df['temperature'].iloc[-1]
                temp_mean = df['temperature'].mean()
                temp_change = temp_current - temp_mean
                temp_trend = "up" if temp_change > 0 else "down"

                hum_current = df['humidity'].iloc[-1]
                hum_mean = df['humidity'].mean()
                hum_change = hum_current - hum_mean
                hum_trend = "up" if hum_change > 0 else "down"

                soil_current = df['soil'].iloc[-1]
                soil_mean = df['soil'].mean()
                soil_change = soil_current - soil_mean
                soil_trend = "up" if soil_change > 0 else "down"
            else:
                temp_current = hum_current = soil_current = 0
                temp_change = hum_change = soil_change = 0
                temp_trend = hum_trend = soil_trend = "stable"

            temp_kpi = gr.HTML(
                create_kpi_card(
                    "Temperature",
                    f"{temp_current:.1f}",
                    "°C",
                    f"{abs(temp_change):.1f}°C",
                    "vs avg",
                    temp_trend,
                    COLORS['temperature']['primary']
                )
            )

            hum_kpi = gr.HTML(
                create_kpi_card(
                    "Humidity",
                    f"{hum_current:.1f}",
                    "%",
                    f"{abs(hum_change):.1f}%",
                    "vs avg",
                    hum_trend,
                    COLORS['humidity']['primary']
                )
            )

            soil_kpi = gr.HTML(
                create_kpi_card(
                    "Soil Moisture",
                    f"{soil_current:.1f}",
                    "%",
                    f"{abs(soil_change):.1f}%",
                    "vs avg",
                    soil_trend,
                    COLORS['soil']['primary']
                )
            )

        # Tab Navigation
        with gr.Tab("📊 Statistics Overview"):
            gr.Markdown("### Detailed Statistical Metrics (Hover over ⓘ for explanations)")
            basic_cards = gr.HTML(create_stat_cards_html(df) if len(df) > 0 else "<p>No data available</p>")

        with gr.Tab("📈 Time Series"):
            ts_explanation = gr.HTML()
            ts_plot = gr.Plot()

        with gr.Tab("📉 Trends & Patterns"):
            hourly_explanation = gr.HTML()
            hourly_plot = gr.Plot()
            gr.Markdown("---")
            daily_explanation = gr.HTML()
            daily_plot = gr.Plot()

        with gr.Tab("🔗 Correlations & Relationships"):
            corr_explanation = gr.HTML()
            corr_plot = gr.Plot()
            gr.Markdown("---")
            scatter_explanation = gr.HTML()
            scatter_plot = gr.Plot()

        with gr.Tab("📊 Distributions"):
            dist_explanation = gr.HTML()
            dist_plot = gr.Plot()

        with gr.Tab("📊 Moving Averages"):
            ma_explanation = gr.HTML()
            var_selector = gr.Radio(
                choices=['temperature', 'humidity', 'soil'],
                value='temperature',
                label="Select Variable for Analysis"
            )
            ma_plot = gr.Plot()

        with gr.Tab("⚠️ Anomaly Detection"):
            gr.HTML(create_explanation_card(
                "Anomaly Detection",
                "Statistical outlier detection using Z-scores. Values with Z-score > 3 are flagged as anomalies (occurring in <0.3% of data if normally distributed).",
                "Anomalies may indicate sensor errors, unusual environmental conditions, or data collection issues. Investigate the cause of flagged measurements.",
                COLORS['status']['critical']
            ))
            anomaly_table = gr.Dataframe(
                label="Detected Anomalies (Z-score > 3)",
                wrap=True
            )

        with gr.Accordion("⚙️ Advanced Settings", open=False):
            with gr.Row():
                refresh_rate = gr.Slider(
                    minimum=5, maximum=300, value=30, step=5,
                    label="Auto-refresh Interval (seconds)"
                )
                alert_threshold = gr.Slider(
                    minimum=15, maximum=35, value=25, step=0.5,
                    label="Temperature Alert Threshold (°C)"
                )

        # Event handlers
        def sync_and_refresh():
            global df
            status, _ = sync_new_data_from_server()
            df = load_data_from_firebase()

            if len(df) > 0:
                info = f"**📊 Current Data:** {len(df)} records | **📅 Range:** {df['timestamp'].min()} to {df['timestamp'].max()}"

                # Update KPIs
                temp_current = df['temperature'].iloc[-1]
                temp_mean = df['temperature'].mean()
                temp_change = temp_current - temp_mean
                temp_trend = "up" if temp_change > 0 else "down"

                hum_current = df['humidity'].iloc[-1]
                hum_mean = df['humidity'].mean()
                hum_change = hum_current - hum_mean
                hum_trend = "up" if hum_change > 0 else "down"

                soil_current = df['soil'].iloc[-1]
                soil_mean = df['soil'].mean()
                soil_change = soil_current - soil_mean
                soil_trend = "up" if soil_change > 0 else "down"

                temp_card = create_kpi_card(
                    "Temperature", f"{temp_current:.1f}", "°C",
                    f"{abs(temp_change):.1f}°C", "vs avg", temp_trend,
                    COLORS['temperature']['primary']
                )

                hum_card = create_kpi_card(
                    "Humidity", f"{hum_current:.1f}", "%",
                    f"{abs(hum_change):.1f}%", "vs avg", hum_trend,
                    COLORS['humidity']['primary']
                )

                soil_card = create_kpi_card(
                    "Soil Moisture", f"{soil_current:.1f}", "%",
                    f"{abs(soil_change):.1f}%", "vs avg", soil_trend,
                    COLORS['soil']['primary']
                )

                return (
                    status, info,
                    temp_card, hum_card, soil_card,
                    create_stat_cards_html(df),
                    *time_series_overview(df),
                    *hourly_patterns(df),
                    *daily_patterns(df),
                    *calculate_correlations(df),
                    *scatter_analysis(df),
                    *distribution_analysis(df),
                    *time_series_decomposition(df, 'temperature'),
                    anomaly_detection(df)
                )

            return (status, "**No data available**") + ("",) * 16

        def update_ma(var):
            if len(df) > 0:
                return time_series_decomposition(df, var)
            return "", go.Figure()

        # Sync button click
        sync_btn.click(
            sync_and_refresh,
            outputs=[
                sync_status, data_info,
                temp_kpi, hum_kpi, soil_kpi,
                basic_cards,
                ts_explanation, ts_plot,
                hourly_explanation, hourly_plot,
                daily_explanation, daily_plot,
                corr_explanation, corr_plot,
                scatter_explanation, scatter_plot,
                dist_explanation, dist_plot,
                ma_explanation, ma_plot,
                anomaly_table
            ]
        )

        # Variable selector for MA
        var_selector.change(
            update_ma,
            inputs=[var_selector],
            outputs=[ma_explanation, ma_plot]
        )

        # Initial load
        if len(df) > 0:
            demo.load(
                lambda: (
                    create_stat_cards_html(df),
                    *time_series_overview(df),
                    *hourly_patterns(df),
                    *daily_patterns(df),
                    *calculate_correlations(df),
                    *scatter_analysis(df),
                    *distribution_analysis(df),
                    *time_series_decomposition(df, 'temperature'),
                    anomaly_detection(df)
                ),
                outputs=[
                    basic_cards,
                    ts_explanation, ts_plot,
                    hourly_explanation, hourly_plot,
                    daily_explanation, daily_plot,
                    corr_explanation, corr_plot,
                    scatter_explanation, scatter_plot,
                    dist_explanation, dist_plot,
                    ma_explanation, ma_plot,
                    anomaly_table
                ]
            )

    return demo

print('✓ Dashboard interface ready!')

✓ Dashboard interface ready!


In [ ]:
# ============== LAUNCH DASHBOARD ==============

print('🚀 Launching Professional Sensor Analytics Dashboard...')
print('='*60)
print('Features:')
print('  ✓ Real-time data sync from server')
print('  ✓ Professional KPI cards with trend indicators')
print('  ✓ Colorblind-safe palettes (Okabe-Ito)')
print('  ✓ Material Design elevation shadows')
print('  ✓ Interactive tooltips with explanations')
print('  ✓ Proper axis labels on all charts')
print('  ✓ Statistical annotations and reference lines')
print('  ✓ Responsive grid layout (8px system)')
print('  ✓ Inter font for professional typography')
print('='*60)

demo = create_dashboard()
demo.launch(debug=True, share=True)  # share=True auto-enabled in Colab

🚀 Launching Professional Sensor Analytics Dashboard...
Features:
  ✓ Real-time data sync from server
  ✓ Professional KPI cards with trend indicators
  ✓ Colorblind-safe palettes (Okabe-Ito)
  ✓ Material Design elevation shadows
  ✓ Interactive tooltips with explanations
  ✓ Proper axis labels on all charts
  ✓ Statistical annotations and reference lines
  ✓ Responsive grid layout (8px system)
  ✓ Inter font for professional typography
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://14d2b9d4a5a127a058.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
